# Case Study: Predicting Wages
## Linear regression using [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) or [patsy]([Patsy](https://patsy.readthedocs.io/en/latest/#) + [statsmodels](http://www.statsmodels.org/stable/index.html#), and cross correlation


In [1]:
# Read the data. I previously copied the data in the .Rdata file to data.csv
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

data = pd.read_csv('data.csv', index_col=False, header=0)
y = data.wage.values
y = y[:, None]					#needs to be 2d for LinearRegression
data.drop(['hsg','wage'],axis=1,inplace=True)
data.describe()

,female,cg,sc,mw,so,we,ne,exp1,exp2,exp3
count,3835.000000,3835.000000,3835.000000,3835.000000,3835.000000,3835.000000,3835.000000,3835.000000,3835.000000,3835.000000
mean,0.417992,0.376271,0.323859,0.287614,0.243546,0.211734,0.257106,13.353194,2.529267,5.812103
std,0.493293,0.484513,0.468008,0.452709,0.429278,0.408591,0.437095,8.639348,2.910554,9.033207
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.040000,0.008000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.360000,0.216000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,1.210000,1.331000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,19.500000,3.802500,7.414875
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.000000,12.250000,42.875000


In [2]:
from scipy.stats import pearsonr
pearsonr(data['ne'],data.mw)
data[(data['ne']==1) & (data['we']==1)]

,female,cg,sc,mw,so,we,ne,exp1,exp2,exp3


### Basic model with 10 features: linear and Quadratic specifications

In [3]:
###  Linear and Quadratic specifications
###  Basic model with 10 features
x = data.values

fmla_basic = LinearRegression(fit_intercept=True)
fmla_basic.fit(x,y)

#Grab the results of the linear fit
n_basic = y.size
p_basic = fmla_basic.coef_.size+fmla_basic.intercept_.size
R2_basic = fmla_basic.score(x,y)
R2adj_basic = 1 - (1-R2_basic)*(n_basic-1)/(n_basic-p_basic-1)
MSEadj_basic = (n_basic/(n_basic-p_basic))*((fmla_basic.predict(x) - y)**2).mean() 	#165.68 in R

print("Regression on basic model: p=%.0f, R2=%.3f, R2adj=%.2f, MSEadj=%.2f" %(p_basic,R2_basic,R2adj_basic,MSEadj_basic))

Regression on basic model: p=11, R2=0.095, R2adj=0.09, MSEadj=165.25


### Flexible model with 33 features: linear and quadratic specifications plus their two-way interactions

In [4]:
# Repeat the linear fit using the same controls plus their interactions
xint=x.copy()
xint=np.delete(xint,0,axis=1)
xint.shape

(3835, 9)

In [5]:
#Calculate columns with interactions of controls
xint=x.copy()
xint=np.delete(xint,0,axis=1)
for i in range(1,x.shape[1]):
	for j in range(i+1,x.shape[1]):
		inter = x[:,i]*x[:,j]
		inter = inter[:,None]
		xint = np.hstack((xint, inter))
female = data.iloc[:,0]; female = female[:, None]
xint=np.hstack((female,xint))

In [6]:
###  Flexible model with 38 features
fmla_flex = LinearRegression(fit_intercept=True)
fmla_flex.fit(xint,y)

#NB: 5 coefficients are not defined in the R code: 
#    sc:cg, mw:so, mw:we, so:we, exp1:exp2

n_flex = y.size
p_flex = fmla_flex.coef_.size+fmla_flex.intercept_.size
R2_flex = fmla_flex.score(xint,y)
R2adj_flex = 1 - (1-R2_flex)*(n_flex-1)/(n_flex-p_flex-1)
MSEadj_flex = (n_flex/(n_flex-p_flex))*((fmla_flex.predict(xint) - y)**2).mean() 	#165.12 in R
print("Regression on flexible model: p=%.0f, R2=%.2f, R2adj=%.2f, MSEadj=%.2f" %(p_flex,R2_flex,R2adj_flex,MSEadj_flex))

Regression on flexible model: p=47, R2=0.10, R2adj=0.09, MSEadj=163.70


In [7]:
### Summary: linear regression on basic and flexible models
print("Regression on basic model:    p=%.0f, R2=%.2f, MSEadj=%.2f" %(p_basic,R2_basic,MSEadj_basic))
print("Regression on flexible model: p=%.0f, R2=%.2f, MSEadj=%.2f" %(p_flex,R2_flex,MSEadj_flex))

Regression on basic model:    p=11, R2=0.10, MSEadj=165.25
Regression on flexible model: p=47, R2=0.10, MSEadj=163.70


### Using cross correlation
### Basic model with 10 features: linear and quadratic specifications with Sample Splitting

In [8]:
### Use cross validation on the data
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,  test_size=0.8, random_state=0)
x_train.shape,y_train.shape

/home/amarin/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((767, 10), (767, 1))

In [9]:
###  Linear and Quadratic specifications
###  Basic model with 10 features
fmla_cvbasic = LinearRegression(fit_intercept=True)
fmla_cvbasic.fit(x_train,y_train)
yhat = fmla_cvbasic.predict(x_test)

# This time let's use sklearn.metrics
from sklearn import metrics
n_cvbasic = yhat.size
p_cvbasic = fmla_cvbasic.coef_.size + fmla_cvbasic.intercept_.size
R2_cvbasic = metrics.r2_score(y_test, yhat)
MSE_cvbasic = metrics.mean_squared_error(y_test, yhat)

print("Regression on basic model using cross validation: p=%.0f, R2=%.2f, MSE=%.2f" %(p_cvbasic, R2_cvbasic, MSE_cvbasic))

Regression on basic model using cross validation: p=11, R2=0.08, MSE=189.35


### Using cross correlation
### Flexible model with 33 features: linear and quadratic specifications plus their two-way interactions

In [10]:
###  Linear and Quadratic specifications plus their two-way interactions
xint_train, xint_test, y_train, y_test = train_test_split(xint, y, test_size=0.8, random_state=0)

fmla_cvflex = LinearRegression(fit_intercept=True)
fmla_cvflex.fit(xint_train,y_train)
yhat_cvflex = fmla_cvflex.predict(xint_test)

n_cvflex = yhat_cvflex.size
p_cvflex = fmla_cvflex.coef_.size + fmla_cvflex.intercept_.size
R2_cvflex = metrics.r2_score(y_test, yhat_cvflex)
MSE_cvflex = metrics.mean_squared_error(y_test, yhat_cvflex)

print("Regression on flexible model using cross validation: p=%.0f, R2=%.2f, MSE=%.2f" %(p_cvflex, R2_cvflex, MSE_cvflex))


Regression on flexible model using cross validation: p=47, R2=0.07, MSE=191.91


In [11]:
print("Regression on basic model using cross validation: p=%.0f, R2=%.2f, MSE=%.2f" %(p_cvbasic, R2_cvbasic, MSE_cvbasic))
print("Regression on flexible model using cross validation: p=%.0f, R2=%.2f, MSE=%.2f" %(p_cvflex, R2_cvflex, MSE_cvflex))

Regression on basic model using cross validation: p=11, R2=0.08, MSE=189.35
Regression on flexible model using cross validation: p=47, R2=0.07, MSE=191.91


# Using patsy and statsmodels
[Patsy](https://patsy.readthedocs.io/en/latest/#): describing statistical models in Python using symbolic formulas

Use pip to install patsy:

`sudo pip install -U patsy`

`sudo pip install -U statsmodels`

Otherwise see [here](https://patsy.readthedocs.io/en/latest/overview.html#download) to download the source patsy-0.4.1.zip and installed it using: 

`sudo python setup.py install`

I installed statsmodel with: 

`sudo python -mpip install statsmodels`

In [12]:
%reset -f

###  Basic model with 10 features
###  Linear and Quadratic specifications

In [13]:
import pandas as pd
import statsmodels.discrete.discrete_model as sm
from patsy import dmatrices
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

data = pd.read_csv('data.csv', index_col=False, header=0)
y, X = dmatrices("wage ~ female + sc+ cg+ mw + so + we + exp1 + exp2 + exp3", 
                 data, return_type = 'dataframe')
fit_basic = sm.OLS(y, X)
result_basic = fit_basic.fit()
yhat_basic = result_basic.predict(X)
result_basic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     44.87
Date:                Fri, 29 Sep 2017   Prob (F-statistic):           3.17e-77
Time:                        18:16:13   Log-Likelihood:                -15235.
No. Observations:                3835   AIC:                         3.049e+04
Df Residuals:                    3825   BIC:                         3.055e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9154      1.299      3.784      0.000       2.368       7.462
female        -1.8264      0.425     -4.302      0.000      -2.659      -0.994
sc             2.4865      0.534      4.654      0.000       1.439       3.534
cg             9.8708      0.562     17.567      0.000       8.769      10.972
mw            -1.2142      0.566     -2.146      0.032      -2.323      -0.105
so             0.4046      0.588      0.688      0.491      -0.748       1.558
we            -0.2508      0.611     -0.410      0.682      -1.449       0.947
exp1           1.0965      0.269      4.077      0.000       0.569       1.624
exp2          -4.0134      1.785     -2.248      0.025      -7.514      -0.513
exp3           0.4603      0.344      1.340      0.180      -0.213       1.134
==============================================================================
Omnibus:                     6626.018   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8721375.158
Skew:                          11.808   Prob(JB):                         0.00
Kurtosis:                     235.426   Cond. No.                         198.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Flexible model with 33 features 
### Linear and Quadratic specifications plus their two-way interactions

In [14]:
y_flex, X_flex = dmatrices("wage ~  female + (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)**2", 
                 data, return_type = 'dataframe')
fit_flex = sm.OLS(y_flex, X_flex)
result_flex = fit_flex.fit()
yhat_flex = result_flex.predict(X_flex)
result_flex.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     13.79
Date:                Fri, 29 Sep 2017   Prob (F-statistic):           5.53e-69
Time:                        18:16:13   Log-Likelihood:                -15217.
No. Observations:                3835   AIC:                         3.050e+04
Df Residuals:                    3802   BIC:                         3.071e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.5524      7.175      2.307      0.021       2.486      30.619
female        -1.8800      0.425     -4.426      0.000      -2.713      -1.047
sc            -2.3865      5.415     -0.441      0.659     -13.003       8.230
cg             2.2405      5.908      0.379      0.705      -9.342      13.823
mw            -5.5194      3.375     -1.635      0.102     -12.137       1.098
so            -2.9144      3.482     -0.837      0.403      -9.742       3.913
we            -0.8054      3.646     -0.221      0.825      -7.953       6.342
exp1          -1.3215      2.073     -0.638      0.524      -5.386       2.743
exp2          12.5218     24.780      0.505      0.613     -36.062      61.106
exp3          -0.0484      0.151     -0.321      0.748      -0.344       0.247
sc:cg       1.071e-13   2.53e-13      0.424      0.672   -3.88e-13    6.02e-13
sc:mw         -0.7226      1.445     -0.500      0.617      -3.555       2.110
sc:so         -0.6513      1.534     -0.424      0.671      -3.659       2.357
sc:we         -0.1047      1.592     -0.066      0.948      -3.227       3.018
sc:exp1        0.8391      1.097      0.765      0.444      -1.311       2.989
sc:exp2       -4.0608      6.512     -0.624      0.533     -16.827       8.706
sc:exp3        0.6330      1.158      0.547      0.585      -1.637       2.904
cg:mw         -0.7609      1.536     -0.496      0.620      -3.772       2.250
cg:so          1.7041      1.569      1.086      0.278      -1.373       4.781
cg:we         -1.4948      1.637     -0.913      0.361      -4.704       1.715
cg:exp1        0.7859      1.245      0.631      0.528      -1.654       3.226
cg:exp2       -0.0490      7.761     -0.006      0.995     -15.265      15.167
cg:exp3       -0.5950      1.462     -0.407      0.684      -3.462       2.272
mw:so      -4.324e-16   5.34e-15     -0.081      0.935   -1.09e-14       1e-14
mw:we       8.422e-15   1.63e-14      0.517      0.605   -2.35e-14    4.04e-14
mw:exp1        1.1076      0.724      1.530      0.126      -0.312       2.527
mw:exp2       -6.0527      4.790     -1.264      0.206     -15.444       3.339
mw:exp3        0.9063      0.919      0.987      0.324      -0.895       2.707
so:we      -7.975e-15    1.5e-14     -0.532      0.595   -3.74e-14    2.14e-14
so:exp1        0.3947      0.748      0.528      0.598      -1.072       1.861
so:exp2       -0.8914      4.973     -0.179      0.858     -10.641       8.858
so:exp3       -0.0352      0.960     -0.037      0.971      -1.917       1.847
we:exp1        0.4719      0.790      0.597      0.550      -1.077       2.021
we:exp2       -3.9188      5.300     -0.739      0.460     -14.309       6.472
we:exp3        0.8050      1.034      0.778      0.436      -1.223       2.833
exp1:exp2     -0.4839      1.505     -0.321      0.748      -3.435       2.467
ex

In [15]:
# Let's use sklearn metrics to summarize
from sklearn import metrics
MSE_basic = metrics.mean_squared_error(y.wage.values, yhat_basic)
R2_basic = metrics.r2_score(y.wage.values, yhat_basic)
print("Using patsy and statsmodels")
print("Regression on basic model: p=%.0f, R2=%.2f, MSE=%.2f" %(10, R2_basic, MSE_basic))

MSE_flex = metrics.mean_squared_error(y_flex.wage.values, yhat_flex)
R2_flex = metrics.r2_score(y_flex.wage.values, yhat_flex)
print("Regression on flexible model: p=%.0f, R2=%.2f, MSE=%.2f" %(33, R2_flex, MSE_flex))

Using patsy and statsmodels
Regression on basic model: p=10, R2=0.10, MSE=165.25
Regression on flexible model: p=33, R2=0.10, MSE=163.70


### Using cross correlation
### Basic model with 10 features: linear and quadratic specifications

In [16]:
### Use cross validation on the data
from sklearn.cross_validation import train_test_split
data_train, data_test = train_test_split(data,  test_size=0.8, random_state=0)
data_train.shape,data_train.shape

((767, 12), (767, 12))

In [17]:
y_trainbasic, X_trainbasic = dmatrices("wage ~ female + sc+ cg+ mw + so + we + exp1 + exp2 + exp3", 
                 data_train, return_type = 'dataframe')
y_testbasic, X_testbasic = dmatrices("wage ~ female + sc+ cg+ mw + so + we + exp1 + exp2 + exp3", 
                 data_test, return_type = 'dataframe')
fit_cvbasic = sm.OLS(y_trainbasic, X_trainbasic)
result_cvbasic = fit_cvbasic.fit()
yhat_cvbasic = result_cvbasic.predict(X_testbasic)
#result_cvbasic.summary()

### Using cross correlation
### Flexible model with 33 features: linear and quadratic specifications plus their two-way interactions

In [18]:
y_trainflex, X_trainflex = dmatrices("wage ~  female + (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)**2", 
                 data_train, return_type = 'dataframe')
y_testflex, X_testflex = dmatrices("wage ~  female + (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)**2", 
                 data_train, return_type = 'dataframe')
fit_cvflex = sm.OLS(y_trainflex, X_trainflex)
result_cvflex = fit_cvflex.fit()
yhat_cvflex = result_cvflex.predict(X_testflex)
result_cvflex.summary();

In [19]:
# Let's use sklearn metrics to summarize
MSE_cvbasic = metrics.mean_squared_error(y_testbasic.wage.values, yhat_cvbasic)
R2_cvbasic = metrics.r2_score(y_testbasic.wage.values, yhat_cvbasic)
print("Using patsy and statsmodels")
print("Regression on basic model using cross validation: p=%.0f, R2=%.2f, MSE=%.2f" %(10, R2_cvbasic, MSE_cvbasic))

MSE_cvflex = metrics.mean_squared_error(y_testflex.wage.values, yhat_cvflex)
R2_cvflex = metrics.r2_score(y_testflex.wage.values, yhat_cvflex)
print("Regression on flexible model using cross validation: p=%.0f, R2=%.2f, MSE=%.2f" %(47, R2_cvflex, MSE_cvflex))

print("uh? did I mess up the last one? oh well..")

Using patsy and statsmodels
Regression on basic model using cross validation: p=10, R2=0.08, MSE=189.35
Regression on flexible model using cross validation: p=47, R2=0.15, MSE=75.50
uh? did I mess up the last one? oh well..
